# JSONEditor JSON编辑器

JSONEditor组件提供了一个可视化编辑器，用于编辑JSON可序列化的数据结构，如Python字典和列表，具有不同编辑模式、插入对象和使用JSON Schema进行验证的功能。

底层实现为`panel.widgets.JSONEditor`，参数基本一致，参考文档：https://panel.holoviz.org/reference/widgets/JSONEditor.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

JSON编辑器提供了一个直观的界面来查看和编辑JSON数据。


In [ ]:
##ignore
import panel as pn
import param

pn.extension('ace', 'jsoneditor')
json_editor = pn.widgets.JSONEditor(value={
    'dict'  : {'key': 'value'},
    'float' : 3.14,
    'int'   : 1,
    'list'  : [1, 2, 3],
    'string': 'A string',
}, width=400)

json_editor

In [3]:
%%vuepy_run --plugins vpanel --show-code --codgen-backend='panel'
<template>
  <PnJSONEditor 
    :width="400"
    v-model="json_data.value"
    @change="on_change"
  />
  <p> {{ json_data.value }} </p>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

json_data = ref({
    'dict'  : {'key': 'value'},
    'float' : 3.14,
    'int'   : 1,
    'list'  : [1, 2, 3],
    'string': 'A string',
})

def on_change(event):
    print(event.new)
</script>

{"vue": "<template>\n  <PnJSONEditor \n    :width=\"400\"\n    v-model=\"json_data.value\"\n    @change=\"on_change\"\n  />\n  <p> {{ json_data.value }} </p>\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\njson_data = ref({\n    'dict'  : {'key': 'value'},\n    'float' : 3.14,\n    'int'   : 1,\n    'list'  : [1, 2, 3],\n    'string': 'A string',\n})\n\ndef on_change(event):\n    print(event.new)\n</script>\n", "setup": ""}



## 编辑模式

JSON编辑器有多种模式，提供不同的查看和编辑`JSONEditor.value`的方式。注意，要启用对`mode='code'`的支持，必须使用`pn.extension('ace')`加载ace编辑器。


In [5]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnCol>
    <PnStaticText value="tree 树形模式" />
    <PnJSONEditor :value="json_data" mode='tree' :width='300' />
  </PnCol>
  
  <PnCol>
    <PnStaticText value="form 表单模式" />
    <PnJSONEditor :value="json_data" mode='form' :width="300" />
  </PnCol>
 
  <PnCol>
    <PnStaticText value="text 文本模式" />
    <PnJSONEditor :value="json_data" mode='text' :width="300" />
  </PnCol>
  
  <PnCol>
    <PnStaticText value="preview 预览模式" />
    <PnJSONEditor :value="json_data" mode='preview' :width="300" />
  </PnCol>
  
  <PnCol>
    <PnStaticText value="view 查看模式" />
    <PnJSONEditor :value="json_data" mode='view' :width="300" />
  </PnCol>
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

json_data = {
    'dict'  : {'key': 'value'},
    'float' : 3.14,
    'int'   : 1,
    'list'  : [1, 2, 3],
    'string': 'A string',
}
</script>

{"vue": "<template>\n  <PnCol>\n    <PnStaticText value=\"tree \u6811\u5f62\u6a21\u5f0f\" />\n    <PnJSONEditor :value=\"json_data\" mode='tree' :width='300' />\n  </PnCol>\n  \n  <PnCol>\n    <PnStaticText value=\"form \u8868\u5355\u6a21\u5f0f\" />\n    <PnJSONEditor :value=\"json_data\" mode='form' :width=\"300\" />\n  </PnCol>\n \n  <PnCol>\n    <PnStaticText value=\"text \u6587\u672c\u6a21\u5f0f\" />\n    <PnJSONEditor :value=\"json_data\" mode='text' :width=\"300\" />\n  </PnCol>\n  \n  <PnCol>\n    <PnStaticText value=\"preview \u9884\u89c8\u6a21\u5f0f\" />\n    <PnJSONEditor :value=\"json_data\" mode='preview' :width=\"300\" />\n  </PnCol>\n  \n  <PnCol>\n    <PnStaticText value=\"view \u67e5\u770b\u6a21\u5f0f\" />\n    <PnJSONEditor :value=\"json_data\" mode='view' :width=\"300\" />\n  </PnCol>\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\njson_data = {\n    'dict'  : {'key': 'value'},\n    'float' : 3.14,\n    'int'   : 1,\n    'list'  : [1, 2,


## 验证

JSONEditor通过提供JSON Schema可以对`value`进行验证。JSON Schema描述了JSON对象必须具有的结构，如必需的属性或值必须具有的类型。更多信息请参见 http://json-schema.org/。


In [7]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnJSONEditor 
    :schema="schema" 
    :value="person_data"
    :height="500"
    :width="400"
  />
</template>
<script lang='py'>
import panel as pn
from vuepy import ref

# 需要初始化jsoneditor扩展
pn.extension('ace', 'jsoneditor')

schema = {
    "title": "Person",
    "type": "object",
    "properties": {
        "firstName": {
            "type": "string",
            "description": "The person's first name."
        },
        "lastName": {
            "type": "string",
            "description": "The person's last name."
        },
        "age": {
            "description": "Age in years which must be equal to or greater than zero.",
            "type": "integer",
            "minimum": 0
        }
    }
}

person_data = {
    'firstName': 2,  # 这将引发验证错误，因为应该是字符串
    'lastName': 'Smith',
    'age': 13.5  # 这将引发验证错误，因为应该是整数
}
</script>

{"vue": "<template>\n  <PnJSONEditor \n    :schema=\"schema\" \n    :value=\"person_data\"\n    :height=\"500\"\n    :width=\"400\"\n  />\n</template>\n<script lang='py'>\nimport panel as pn\nfrom vuepy import ref\n\n# \u9700\u8981\u521d\u59cb\u5316jsoneditor\u6269\u5c55\npn.extension('ace', 'jsoneditor')\n\nschema = {\n    \"title\": \"Person\",\n    \"type\": \"object\",\n    \"properties\": {\n        \"firstName\": {\n            \"type\": \"string\",\n            \"description\": \"The person's first name.\"\n        },\n        \"lastName\": {\n            \"type\": \"string\",\n            \"description\": \"The person's last name.\"\n        },\n        \"age\": {\n            \"description\": \"Age in years which must be equal to or greater than zero.\",\n            \"type\": \"integer\",\n            \"minimum\": 0\n        }\n    }\n}\n\nperson_data = {\n    'firstName': 2,  # \u8fd9\u5c06\u5f15\u53d1\u9a8c\u8bc1\u9519\u8bef\uff0c\u56e0\u4e3a\u5e94\u8be5\u662f\u5b57\u7b26\u


## API

### 属性

| 属性名        | 说明                                                                 | 类型                          | 默认值  |
|-------------|--------------------------------------------------------------------|-----------------------------|--------|
| disabled    | 是否禁用编辑器（等同于设置 mode='view'）                               | ^[bool]                     | False  |
| menu        | 是否显示主菜单栏（包含格式、排序、转换、搜索等功能）                        | ^[bool]                     | True   |
| mode        | 编辑器模式：'view'(只读)、'form'(仅值可修改)、'tree'(树)、'text'(纯文本)、'preview'(预览大文件) | ^[str]                      | 'tree' |
| search      | 是否在右上角显示搜索框（仅在 tree/view/form 模式下可用）                   | ^[bool]                     | True   |
| schema      | 用于验证JSON数据的JSON模式（定义必需属性和值类型等）                       | ^[dict]                     | None   |
| value       | 当前可编辑的JSON数据结构                                               | ^[str]                      | ""     |

### Events

| 事件名 | 说明 | 类型 |
| --- | --- | --- |
| change | 当JSON数据改变时触发 | ^[Callable]`(event: dict) -> None` |

### Slots

| 插槽名 | 说明 |
| --- | --- |
| | |

### 方法

| 属性名 | 说明 | 类型 |
| --- | --- | --- |
| | | |


In [8]:
##ignore
import panel as pn
import param

pn.extension('ace', 'jsoneditor')

json_editor = pn.widgets.JSONEditor(value={
    'dict'  : {'key': 'value'},
    'float' : 3.14,
    'int'   : 1,
    'list'  : [1, 2, 3],
    'string': 'A string',
}, width=400)

pn.Row(json_editor.controls(jslink=True), json_editor)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] LiteralInput(description='JSON data to be edited.', name='Value', serializer='json', value={'dict': {'key': 'value'},...})
            [2] Checkbox(name='Disabled')
            [3] Checkbox(name='Menu', value=True)
            [4] Select(description="Sets the editor mode. In ..., name='Mode', options=OrderedDict([('tree', ...]), value='tree')
            [5] Checkbox(name='Search', value=True)
            [6] ListInput(description='Current selection.', name='Selection', serializer='json', type=<class 'list'>)
            [7] DictInput(description='Validate the JSON object ..., name='Schema', serializer='json', type=<class 'dict'>)
            [8] ListInput(description='Array of templates that w..., name='Templates', serializer='json', type=<class 'list'>)
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
            [2] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
            [3] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
            [4] IntInput(description='The height of the compone..., name='Height', start=0)
            [5] IntInput(description='Minimal width o..., name='Min width', start=0)
            [6] IntInput(description='Minimal height o..., name='Min height', start=0)
            [7] IntInput(description='Maximum width o..., name='Max width', start=0)
            [8] IntInput(description='Maximum height o..., name='Max height', start=0)
            [9] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
            [10] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
            [11] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
            [12] IntInput(description='The width of the componen..., name='Width', start=0, value=400)
            [13] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]))
            [16] Checkbox(name='Visible', value=True)
    [1] JSONEditor(value={'dict': {'key': 'value'},...}, width=400)